In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd "/content/drive/My Drive/Colab Notebooks/CoE202/Collaborative Filtering"

## Collaborative Filtering (CF)

<figure class="image">
  <img src="https://drive.google.com/uc?export=view&id=185fQI_jd3DewJRSKO7TEIZH6Sjr15UGc" width="50%" height="50%" title="recommender system" alt="recommender system"></img>
</figure>

- The most prominent approach to generate recommendations
    - Used by large, commercial e-commerce sites
    - Well-understood, various algorithms and variations exist
    - Applicable in many domains
- Use the **wisdom of the crowd** to recommend items
- Basic assumption and idea
    - Users give ratings to items (implicitly or explicitly)
    - <span style="color:red">**Customers who had similar tastes in the past will have similar tastes in the future**</span>

### Neighborhood-based CF
- Main idea
    - Similar users display similar patterns of rating behavior (<span style="color:red">**User-based CF**</span>)
    - Similar items receive similar ratings (<span style="color:red">**Item-based CF**</span>)
- How do we define similarity between users and items?
    - We define similarity between users in terms of items they purchased!
    - We define similarity between items in terms of users who purchased them!
    - We learned variety of similarity measures in the lecture (e.g., Euclidean distance, Jaccard similarity, Cosine similarity, Pearson correlation)

### Model-based CF
#### Matrix Factorization
As we learned in the lecture, we can factorize **ratings matrix** into **user matrix** and **item matrix**.

<figure class="image">
  <img src="https://drive.google.com/uc?export=view&id=1880BHOvpFW66QjjjnnN-exW_HOyX9EkU" width="50%" height="50%" title="recommender system" alt="recommender system"></img>
</figure>

Ratings can be interpreted as **dot product** of user latent and item latent.  
So we can obtain user/item matrix by decomposing ratings matrix,  
and predict unobserved ratings with obtained user/item matrix.

Okay then, how can we decompose the ratings matrix into user and item latent matrix?

**Singular Value Decomposition (SVD)** is a famous linear algebra technique for matrix factorization.  
And it is often used for **dimensionality reduction**.  

If you are not familiar with SVD, please refer to basic linear algebra class or chapter 4 in below linked book. (https://mml-book.github.io/book/mml-book.pdf)


However, applying SVD to recommender system have the following issues.
- Predicted values are often negative.
- Zero replacement decreases prediction quality.
    - <span style="color:red">The meaning of "zero" is different from that of "unknown"</span>.
    - We should be careful whether we want to set missing values to zero.


So, we can model directly leveraging **only observed ratings**, while **avoiding overfitting** through an adequate regularized model, such as :  
$min \frac{1}{2} \sum_{(u, i)\in R}{(r_{ui} -\mu -b_{u}^{user} -b_{i}^{item} -p_{u}q_{i}^{T})^{2}} + \lambda(|p_{u}|^{2} + |q_{i}|^{2} + {b_{u}^{user}}^{2} + {b_{i}^{item}}^{2})$  
where $p_{u}$ and $q_{i}$ are the latent factor of user $u$ and item $i$, respectively.  
$b_{u}$ and $b_{i}$ are the bias term of user $u$ and item $i$, respectively.  
$\mu$ is the mean of the observed ratings matrix.  


A simple **gradient descent** technique can be applied to solve the equation.

#### Gradient Descent

In optimization problem, we have two approaches to find solution of the problem.  
First, we can simply solve the problem in **closed form**.  
However, in the case of complex function, it may not be feasible to solve in the closed form.  
To address the issue, we can iteratively search the solution space improving the target value. We call such a method as **improving search**.

Gradient descent is a first-order iterative optimization algorithm for finding local minimum of a differentiable function.  
The idea is to take repeated steps in the **opposite direction of the gradient** of the function at the current point, because this is the direction of steepest descent.


<figure class="image">
  <img src="https://drive.google.com/uc?export=view&id=18o3tBBj9roZsHkLXdatayiGc3faF0zrM" width="50%" height="50%" title="recommender system" alt="recommender system"></img>
</figure>

If gradient > 0 , then increasing the weight will increase the cost function.  
If gradient < 0 , then increasing the weight will decrease the cost function.

So we update the weight (parameters) with the partial derivative of the cost.  
$w \leftarrow w - \alpha \frac{\partial Cost}{\partial w}$



Gradient descent is very general algorithm which is very **effective** and **scalable**.  
However, we should use gradient descent taking the below in regard.
- **Local minima**
    - Sensitive to the starting point
<figure class="image">
  <img src="https://drive.google.com/uc?export=view&id=18loEZYzhQAZj0A6192fwIGZ0M2rHeXUO" width="40%" height="40%" title="recommender system" alt="recommender system"></img>
</figure>
- **Learning rate**
    - Too large? Too small?
<figure class="image">
  <img src="https://drive.google.com/uc?export=view&id=18ZvvZVl3oTAo9mzILltbE-S-3Cq_a3Yt" width="40%" height="40%" title="recommender system" alt="recommender system"></img>
</figure>

#### Matrix Factorization

<figure class="image">
  <img src="https://drive.google.com/uc?export=view&id=186dDOMvTxDXGrVqW1kke74zMQT3DvE9e" width="70%" height="70%" title="recommender system" alt="recommender system"></img>
</figure>

We can minimize the below cost function with gradient descent.  
$min \frac{1}{2} \sum_{(u, i)\in R}{(r_{ui} -\mu -b_{u}^{user} -b_{i}^{item} -p_{u}q_{i}^{T})^{2}} + \lambda(|p_{u}|^{2} + |q_{i}|^{2} + {b_{u}^{user}}^{2} + {b_{i}^{item}}^{2})$  
$error = (r_{ui} -\mu -b_{u}^{user} -b_{i}^{item} -p_{u}q_{i}^{T})$


- Compute the partial derivative of given cost function respect to $p_{u}$ and $q_{i}$.  
$\frac{\partial cost}{\partial p_{u}} = -error * q_{i} + \lambda p_{u}$  
$\frac{\partial cost}{\partial q_{i}} = -error * p_{u} + \lambda q_{i}$  
$\frac{\partial cost}{\partial b_{u}^{user}} = -error + \lambda b_{u}^{user}$  
$\frac{\partial cost}{\partial b_{i}^{item}} = -error + \lambda b_{i}^{item}$  


- Update the latent of the user and item.  
$p_{u} \leftarrow p_{u} - \alpha(-error * q_{i} + \lambda p_{u})$  
$q_{i} \leftarrow q_{i} - \alpha(-error * p_{u} + \lambda q_{i})$  
$b_{u}^{user} \leftarrow b_{u}^{user} - \alpha(-error + \lambda b_{u}^{user})$  
$b_{i}^{item} \leftarrow b_{i}^{item} - \alpha(-error + \lambda b_{i}^{item})$

In [ ]:
import numpy as np
import data
from timeit import default_timer as timer

In [ ]:
class SVD():

    def __init__(self, train, test, k, learning_rate, reg_param, epochs, verbose = False):
        """
        param R : Rating Matrix
        param k : latent parameter
        param learning_rate : alpha on weight update
        param reg_param : regularization parameter
        param epochs : training epochs
        param verbose : print status
        """
        
        self.R = train
        self.test = test
        self.num_users, self.num_items = train.shape
        self.k = k
        self.learning_rate = learning_rate
        self.reg_param = reg_param
        self.epochs = epochs
        self.verbose = verbose
        
    
    def fit(self):
        """
        training Matrix Factorization : update matrix latent weight and bias
        """
        # init latent features
        self.P = np.random.normal(scale = 1.0/self.k, size = (self.num_users, self.k))
        self.Q = np.random.normal(scale = 1.0/self.k, size = (self.num_items, self.k))
        
        # init biases
        self.b_P = np.zeros(self.num_users)
        self.b_Q = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        self.training_process = []
        
        start = timer()
        
        for epoch in range(self.epochs):
            for u in range(self.num_users):
                for i in range(self.num_items):
                    if self.R[u, i] > 0:
                        self.gradient_descent(u, i, self.R[u, i])
            
            train_cost, test_cost = self.cost()
            self.training_process.append((epoch, train_cost, test_cost))
            
            if self.verbose == True and ((epoch + 1) % 10 == 0 ):
                print("Iteration : %d, train_cost = %.4f, test_cost = %.4f" % (epoch+1, train_cost, test_cost))
        
        print("time : %.4f sec" % (timer() - start) )
        
    
    def cost(self):
        """
        compute RMSE
        """
        xi, yi = self.R.nonzero()
        test_x, test_y = self.test.nonzero()
        predicted = self.get_complete_matrix()
        cost_train = 0
        cost_test = 0
        
        for x, y in zip(xi, yi):
            cost_train += pow(self.R[x, y] - predicted[x, y], 2)
        
        for i, j in zip(test_x, test_y):
            cost_test += pow(self.test[i, j] - predicted[i, j], 2)
        
        return np.sqrt(cost_train/len(xi)), np.sqrt(cost_test/len(test_x))
        
    
    def gradient(self, error, u, i):
        """
        gradient of latent feature for GD
        param error : rating - prediction error
        param u : user index
        param i : item index
        """
        dp = (error * self.Q[i, :]) - (self.reg_param * self.P[u, :])
        dq = (error * self.P[u, :]) - (self.reg_param * self.Q[i, :])
        
        return dp, dq
    
    
    def gradient_descent(self, u, i, rating):
        """
        gradient descent function
        param u : user index
        param i : item index
        param rating : rating of (u, i)
        """
        
        prediction = self.get_prediction(u,i)
        error = rating - prediction
        
        self.b_P[u] += self.learning_rate * (error - self.reg_param * self.b_P[u])
        self.b_Q[i] += self.learning_rate * (error - self.reg_param * self.b_Q[i])
        
        dp, dq = self.gradient(error, u, i)
        self.P[u, :] += self.learning_rate * dp
        self.Q[i, :] += self.learning_rate * dq
        
    
    def get_prediction(self, u, i):
        """
        get predicted rating by user i on item j
        """
        return self.b + self.b_P[u] + self.b_Q[i] + self.P[u, :].dot(self.Q[i, :].T)

    
    def get_complete_matrix(self):
        """
        computer complete matrix
        """
        return self.b + self.b_P[:, np.newaxis] + self.b_Q[np.newaxis,:] + self.P.dot(self.Q.T)
    
    
    def print_results(self):
        """
        print fit results
        """

        print("Final R matrix:")
        print(self.get_complete_matrix())
        print("\n")
        print("Final RMSE: {:.4f}".format(self.training_process[self.epochs-1][2]))

In [ ]:
train = data.train
test = data.test

factorizer = SVD(train, test, k=40, learning_rate=0.01, reg_param=0.01, epochs=30, verbose=True)
factorizer.fit()
factorizer.print_results()